In [34]:
from pymc import Model, Binomial, Normal, MvNormal, math, Exponential, Beta, sample
import numpy as np
import pandas as pd
import pytensor.tensor as pt
from pytensor.tensor.linalg import cholesky
from utility import data_loader, data_cleaner, sample_dist
import arviz

In [10]:
berlin_data, vanautu_data, adult_data, child_data = data_loader()

In [11]:
d_berlin, d_vanautu, MA, pop_berlin, pop_berlin_raw, pop_vanautu_raw, pop_vanautu = data_cleaner(berlin_data, vanautu_data, adult_data, child_data)

/home/harrison/Desktop/gitHubRepos/CausalAnalysis/utility.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adult_data["choice"] = adult_data["T1_ad_choice_1yes"]
/home/harrison/Desktop/gitHubRepos/CausalAnalysis/utility.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_data["choice"] = child_data["T1_choice_1yes"]
/home/harrison/Desktop/gitHubRepos/CausalAnalysis/utility.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

In [12]:
# Create matrices with zeros
sample_berlin, sample_vanuatu = sample_dist(d_berlin, d_vanautu)

In [13]:
#Covaraince Kenrel defined for the paper and implemented in Stan.
MA = 20
age = d_berlin["Age_binned"].values - 1
outcome = d_berlin["choice"]
gender = d_berlin["gender"].values - 1
d_berlinf = d_berlin[d_berlin["gender"] == 1]
d_berlinm =d_berlin[d_berlin["gender"] == 2]
d_berlinf['Age_binned'] = d_berlinf["Age_binned"].astype(int)
d_berlinm['Age_binned'] = d_berlinm["Age_binned"].astype(int)

/tmp/ipykernel_200765/3536775231.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_berlinf['Age_binned'] = d_berlinf["Age_binned"].astype(int)
/tmp/ipykernel_200765/3536775231.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_berlinm['Age_binned'] = d_berlinm["Age_binned"].astype(int)


In [32]:
P_pop = pop_berlin_raw
P_other = pop_vanautu_raw


def GPL(K, C, D, S, epsilon=1e-6):
    KR = np.arange(1, K + 1)
    Rho = C * pt.exp(-D * ((KR[:, None] - KR[None, :]) ** 2) / K ** 2)
    pt.fill_diagonal(Rho, 1)
    
    # Add a small diagonal perturbation
    Rho += epsilon * pt.eye(K)
    
    return S * cholesky(Rho)
    
with Model() as gp:
    # Priors for parameters
    alpha = Normal('alpha', mu=0, sigma=3, shape=2)
    eta = Exponential('eta', lam=2, shape=2)
    sigma = Exponential('sigma', lam=1, shape=2)
    rho = Beta('rho', alpha=10, beta=1, shape=2)

    # Gaussian process age effects
    age_effectf = MvNormal("age_effectf",
        mu=np.zeros(MA),
        chol=GPL(MA, rho[0], eta[0], sigma[0])#, shape = MA
    ) 

    age_effectm = MvNormal("age_effectm",
        mu=np.zeros(MA),
        chol=GPL(MA, rho[1], eta[1], sigma[1])#, shape = MA
    ) 
      
    # Linear model
    pf = math.invlogit(alpha[0] + age_effectf[ np.array(d_berlinf['Age_binned']) ] ) 
    pm = math.invlogit(alpha[1] + age_effectm[ np.array(d_berlinm['Age_binned']) ] ) 
    
    # Likelihood function
    outcome_obsf = Binomial('outcome_obsf', n=1, p = pf, observed = d_berlinf["choice"].values)
    outcome_obsm = Binomial('outcome_obsm', n=1, p = pm, observed = d_berlinm["choice"].values)

In [ ]:
with gp:
     trace = sample(5000, tune=1000, chains=4, cores=4, init='adapt_diag', target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, eta, sigma, rho, age_effectf, age_effectm]


In [35]:
arviz.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],-0.111,0.315,-0.790,0.427,0.066,0.047,23.0,41.0,1.04
alpha[1],0.497,0.453,-0.195,1.246,0.231,0.197,4.0,14.0,1.47
age_effectf[0],-0.117,0.259,-0.472,0.398,0.081,0.059,10.0,40.0,1.15
age_effectf[1],-0.091,0.243,-0.438,0.384,0.082,0.060,9.0,32.0,1.17
age_effectf[2],-0.064,0.228,-0.390,0.373,0.084,0.062,8.0,32.0,1.19
age_effectf[3],-0.036,0.215,-0.348,0.369,0.085,0.063,7.0,31.0,1.23
age_effectf[4],-0.006,0.203,-0.299,0.388,0.087,0.065,6.0,18.0,1.29
age_effectf[5],0.024,0.193,-0.240,0.418,0.088,0.066,5.0,16.0,1.38
age_effectf[6],0.055,0.185,-0.189,0.428,0.089,0.068,4.0,16.0,1.48
age_effectf[7],0.086,0.178,-0.144,0.461,0.091,0.070,4.0,16.0,1.62
